In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from scipy.optimize import minimize
import math

In [3]:
data1 = pd.read_csv("E:/Fall 19/Projects/Project 2/project2.conversion.7.csv")
data1.head()

,renewal_date,status,Service.Agreement.Id,Chan.Prof
0,20130501.0,Loss,101318541038,Direct
1,20130601.0,Loss,101368799128,Direct
2,20130601.0,Loss,102254215870,Direct
3,20130601.0,Loss,102274104936,Direct
4,20130601.0,Loss,102274105019,Direct


In [4]:
data2 = pd.read_csv("E:/Fall 19/Projects/Project 2/project2.incident.7.csv")
data2.head()

,calendar_yearmonth,Year_Name,Month_Name,SAID,number_of_Cases,number_of_escalation,number_of_parts_used,number_of_single_visit_missed,number_of_response_missed
0,201202,2012,2,102378111031,1,0,0,0,0
1,201207,2012,7,102378111031,1,0,1,0,0
2,201201,2012,1,102378111031,2,0,2,0,0
3,201210,2012,10,102379259014,1,0,0,0,0
4,201202,2012,2,102379259014,1,0,0,0,0


In [5]:
print(data1.isnull().sum())
print(data2.isnull().sum())

renewal_date            16
status                   0
Service.Agreement.Id     0
Chan.Prof                0
dtype: int64
calendar_yearmonth               0
Year_Name                        0
Month_Name                       0
SAID                             0
number_of_Cases                  0
number_of_escalation             0
number_of_parts_used             0
number_of_single_visit_missed    0
number_of_response_missed        0
dtype: int64


In [6]:
data1 = data1.sort_values(by = ['renewal_date', 'Service.Agreement.Id'], ascending=True)
data1 = data1.drop_duplicates(keep = 'last')
data1 = data1.dropna()
data1.head()

,renewal_date,status,Service.Agreement.Id,Chan.Prof
11542,20130101.0,Converted,103552674212,Direct
14958,20130201.0,Loss,103765758473,Direct
11388,20130301.0,Loss,103429937262,Direct
12069,20130301.0,Loss,103659475560,Direct
6219,20130302.0,Converted,102551146142,Direct


In [7]:
print(data1.isnull().sum())

renewal_date            0
status                  0
Service.Agreement.Id    0
Chan.Prof               0
dtype: int64


In [8]:
data2.duplicated().any()

False

In [9]:
data1['renewal_date'] = pd.to_datetime(data1['renewal_date'], format='%Y%m%d')
data1.head()

,renewal_date,status,Service.Agreement.Id,Chan.Prof
11542,2013-01-01,Converted,103552674212,Direct
14958,2013-02-01,Loss,103765758473,Direct
11388,2013-03-01,Loss,103429937262,Direct
12069,2013-03-01,Loss,103659475560,Direct
6219,2013-03-02,Converted,102551146142,Direct


In [10]:
data2['calendar_yearmonth'] = pd.to_datetime(data2['calendar_yearmonth'], format='%Y%m')
data3 = data2.iloc[:, [0, 3, 4]].copy()
data3.head()

,calendar_yearmonth,SAID,number_of_Cases
0,2012-02-01,102378111031,1
1,2012-07-01,102378111031,1
2,2012-01-01,102378111031,2
3,2012-10-01,102379259014,1
4,2012-02-01,102379259014,1


In [12]:
data4 = data1.merge(data3, left_on = 'Service.Agreement.Id', right_on = 'SAID')
data4 = data4.drop(columns = 'SAID')
diff = data4['renewal_date']-data4['calendar_yearmonth']
index = diff > pd.Timedelta(0,'m')
data4['time_dif'] = diff
data4 = data4[index]
data4['status'] = data4['status'].map({'Converted':1, 'Loss':0})
data4.index = range(len(data4['status']))
data4['time_dif'] = data4['time_dif']/ np.timedelta64(1, 'D')
data4.head(10)
data4.to_csv('merge_data.csv', columns=['status','Chan.Prof', 'number_of_Cases', 'time_dif'], index=False)

In [146]:
data4 = data4.groupby(by='Service.Agreement.Id')
data4.get_group(103765758473)

,renewal_date,status,Service.Agreement.Id,Chan.Prof,calendar_yearmonth,number_of_Cases,time_dif
5,2013-02-01,1,103765758473,Direct,2011-11-01,1,458.0
6,2013-02-01,1,103765758473,Direct,2012-01-01,2,397.0
7,2013-02-01,1,103765758473,Direct,2012-06-01,1,245.0


In [183]:
print(i[0])
#print(np.ndenumerate(data4.get_group(i[0])).index)
j = data4.get_group(i[0]).index[0]
data4['status']
#data.index

101261727583


In [178]:
for i in (data4['Service.Agreement.Id'].unique()):
    calc=[]
    for j in data4.get_group(i[0]).index:
        calc.append((math.exp(0.01*(data4['time_dif'][j]))) * (math.log(1+data4['number_of_Cases'][j])))
    print(i[0], sum(calc))

Exception: Column(s) time_dif already selected

In [117]:
data5 = pd.DataFrame(np.nan, index = range((len(data4['Service.Agreement.Id'].unique()))), columns = ['sum', 'ID'])

In [137]:
count = -1
start = 0
for _ in range(len(data4['Service.Agreement.Id'])):
    calc = []
    count +=1
    data5['ID'][count] = data4['Service.Agreement.Id'][start]
    for i in range(len(data4['Service.Agreement.Id'])):
        if data4['Service.Agreement.Id'][start] == data4['Service.Agreement.Id'][i]:
            calc.append((math.exp(0.01*(data4['time_dif'][i]))) * (math.log(1+data4['number_of_Cases'][i])))
            data5['sum'][count] = sum(calc)              
        else:
            start = start + i
            break

In [139]:
data5.head()

,sum,ID
0,24.383731,1.035527e+11
1,NaN,1.037658e+11
2,NaN,1.037658e+11
3,NaN,1.037658e+11
4,NaN,1.037658e+11


In [24]:
cases = data4['number_of_Cases']
time_diff = data4['time_dif']

In [32]:
def full_model(coff):
    b0, b1, r = coff
    P = []
    Y = []
    e = []
    for i in range(len(cases)):
        calc = ((math.exp(r*(time_diff[i]))) * (math.log(1+cases[i])))
        Z = b0 + b1 * (calc)
        P.append(1/(1+(math.exp(-Z))))
        Y.append(1 if P[i]>0.5 else 0)
        e.append((Y[i]*math.log(1+P[i]))+((1-Y[i])*math.log(1-P[i]+1)))
        value = -1*sum(e)
    return value

In [33]:
p0 = 0
p1 = 0
r1 = 0
bound = ((None, None), (None, None), (None, None))
opt = minimize(full_model, (p0, p1, r1), method = "L-BFGS-B", bounds = bound)
opt

KeyError: 5

In [11]:
a0 = 1
a1 = 0.02
n=1000
X = np.random.randint(-25, 25, size = n)
Y_S = []
P_S = []
for j in range(n):
    Z_S = a0 + a1*X[j]
    P_S.append(1/(1+(math.exp(-Z_S))))
    Y_S.append(1 if P_S[j]>0.5 else 0)

In [12]:
def func(coff):
    b0, b1 = coff
    P = []
    Y = []
    e=[]
    for i in range(len(X)):
        #print('X=',X[i])
        Z = b0+b1*X[i]
        #print('Z=',Z)
        P.append(1/(1+(math.exp(-Z))))
        #print('P=',P[i])
        Y.append(1 if P[i]>0.5 else 0)
        #print('Y=',Y[i])
        e.append((Y[i]*math.log(1+P[i]))+((1-Y[i])*math.log(1-P[i]+1)))
        #print('e =',e[i])
    value = -1*sum(e)
    #print('sum =',value)
    return value

In [13]:
p0 = 0
p1 = 0
bound = ((None, None), (None, None))
opt = minimize(func, (p0,p1), method = "L-BFGS-B", bounds = bound)
opt

      fun: -693.1471626974562
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0., 0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 132
      nit: 21
   status: 0
  success: True
        x: array([13.65847449, 27.61849279])

In [ ]:
1/(1+(math.exp(990)))

In [ ]:
Z = b0+b1*X[i]